# Week 9 - Part 1: Advanced Business Dashboards

**Course:** Python Data Analysis for Business Intelligence  
**Week:** 9 | **Session:** Thursday | **Part:** 1 of 3  
**Duration:** 20 minutes | **Date:** June 5, 2025

## Learning Objectives
By the end of this session, you will be able to:
- Design multi-page business applications with navigation
- Implement advanced layout patterns for executive dashboards
- Create role-based user interfaces for different stakeholders
- Build responsive dashboards that work across devices
- Apply professional styling and branding to business applications

---

## 🎯 Business Context: Enterprise Dashboard Architecture

**Real-World Challenge**: Olist's executive team needs a comprehensive business intelligence platform that serves multiple stakeholder groups:

### Stakeholder Requirements:
- **🏢 C-Level Executives**: High-level KPIs, strategic metrics, board-ready visualizations
- **📊 Department Managers**: Operational metrics, team performance, drill-down capabilities
- **👨‍💼 Analysts**: Detailed data exploration, custom filtering, export capabilities
- **🎯 Sales Teams**: Territory performance, quota tracking, customer insights

**Architecture Challenge**: 
- Single application serving multiple user types
- Consistent branding and navigation
- Scalable design for future feature additions
- Mobile-responsive interface for on-the-go access

**Today's Solution**: Multi-page Streamlit architecture with role-based navigation and professional enterprise design.

---

## 🛠️ Setup: Advanced Environment Configuration

Let's set up our advanced development environment:

In [ ]:
# Advanced imports for enterprise dashboards
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import time
import base64
from io import BytesIO

# Course utilities
from Utilities.visualization_helper import set_plotting_style
from Utilities.olist_helper import load_olist_data, join_order_data
from Utilities.colab_helper import setup_colab

# Setup
plt, sns = set_plotting_style()
setup_colab()

print("✅ Advanced dashboard environment ready!")
print("🏢 Ready to build enterprise-grade business intelligence applications")

## 🏗️ Section 1: Multi-Page Application Architecture (8 minutes)

Let's create a comprehensive multi-page business intelligence platform:

In [ ]:
%%writefile enterprise_dashboard_app.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import time

# Configure page with enterprise settings
st.set_page_config(
    page_title="Olist Business Intelligence Platform",
    page_icon="🏢",
    layout="wide",
    initial_sidebar_state="expanded",
    menu_items={
        'Get Help': 'https://olist.com/support',
        'Report a bug': 'https://olist.com/bug-report',
        'About': "Olist Business Intelligence Platform v2.1"
    }
)

# Custom CSS for enterprise branding
st.markdown("""
<style>
    /* Corporate Header */
    .main-header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 2rem;
        border-radius: 10px;
        color: white;
        margin-bottom: 2rem;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    
    /* Navigation Styling */
    .nav-container {
        background: #f8f9fa;
        padding: 1rem;
        border-radius: 8px;
        border-left: 4px solid #667eea;
        margin-bottom: 1rem;
    }
    
    /* KPI Cards */
    .kpi-card {
        background: white;
        padding: 1.5rem;
        border-radius: 8px;
        border: 1px solid #e0e0e0;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        margin: 0.5rem 0;
    }
    
    /* Status Indicators */
    .status-green {
        background: #d4edda;
        border: 1px solid #c3e6cb;
        color: #155724;
        padding: 0.75rem;
        border-radius: 5px;
        border-left: 4px solid #28a745;
    }
    
    .status-yellow {
        background: #fff3cd;
        border: 1px solid #ffeaa7;
        color: #856404;
        padding: 0.75rem;
        border-radius: 5px;
        border-left: 4px solid #ffc107;
    }
    
    .status-red {
        background: #f8d7da;
        border: 1px solid #f5c6cb;
        color: #721c24;
        padding: 0.75rem;
        border-radius: 5px;
        border-left: 4px solid #dc3545;
    }
    
    /* Chart Containers */
    .chart-container {
        background: white;
        padding: 1rem;
        border-radius: 8px;
        border: 1px solid #e0e0e0;
        margin: 1rem 0;
    }
    
    /* Footer */
    .footer {
        text-align: center;
        color: #666;
        padding: 2rem;
        border-top: 1px solid #e0e0e0;
        margin-top: 3rem;
    }
</style>
""", unsafe_allow_html=True)

# Session state initialization
if 'user_role' not in st.session_state:
    st.session_state.user_role = 'Executive'
if 'selected_page' not in st.session_state:
    st.session_state.selected_page = 'Dashboard Overview'

# Generate comprehensive business data
@st.cache_data(ttl=600)  # Cache for 10 minutes
def generate_enterprise_data():
    """
    Generate comprehensive business data for enterprise dashboard.
    """
    np.random.seed(42)
    
    # Generate 6 months of daily data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=180)
    date_range = pd.date_range(start_date, end_date, freq='D')
    
    # Simulate business growth and seasonality
    base_orders = 1000
    growth_trend = np.arange(len(date_range)) * 2
    seasonal = np.sin(np.arange(len(date_range)) * 2 * np.pi / 30) * 100  # Monthly cycle
    weekly = np.sin(np.arange(len(date_range)) * 2 * np.pi / 7) * 50      # Weekly cycle
    noise = np.random.normal(0, 50, len(date_range))
    
    daily_orders = base_orders + growth_trend + seasonal + weekly + noise
    daily_orders = np.maximum(daily_orders, 100)  # Minimum orders
    
    # Generate correlated business metrics
    revenue_per_order = np.random.normal(85, 15, len(date_range))
    daily_revenue = daily_orders * revenue_per_order
    
    customer_satisfaction = np.random.normal(4.2, 0.4, len(date_range))
    customer_satisfaction = np.clip(customer_satisfaction, 1, 5)
    
    # Create main DataFrame
    df = pd.DataFrame({
        'date': date_range,
        'orders': daily_orders.astype(int),
        'revenue': daily_revenue,
        'customer_satisfaction': customer_satisfaction,
        'new_customers': (daily_orders * np.random.uniform(0.3, 0.7, len(date_range))).astype(int),
        'returning_customers': (daily_orders * np.random.uniform(0.3, 0.7, len(date_range))).astype(int),
        'conversion_rate': np.random.normal(3.8, 0.6, len(date_range)),
        'avg_order_value': revenue_per_order
    })
    
    # Generate department-specific data
    departments = ['Sales', 'Marketing', 'Operations', 'Customer Service', 'Finance']
    dept_data = []
    
    for dept in departments:
        performance = np.random.uniform(0.7, 1.3, len(date_range))
        target_achievement = np.random.normal(95, 10, len(date_range))
        
        dept_df = pd.DataFrame({
            'date': date_range,
            'department': dept,
            'performance_score': performance,
            'target_achievement': np.clip(target_achievement, 60, 130),
            'team_size': np.random.randint(5, 25),
            'budget_utilization': np.random.uniform(0.8, 1.1, len(date_range))
        })
        dept_data.append(dept_df)
    
    dept_df = pd.concat(dept_data, ignore_index=True)
    
    return df, dept_df

# Load enterprise data
main_df, dept_df = generate_enterprise_data()

# Corporate Header
st.markdown("""
<div class="main-header">
    <h1>🏢 Olist Business Intelligence Platform</h1>
    <p>Enterprise Dashboard • Real-time Analytics • Strategic Insights</p>
</div>
""", unsafe_allow_html=True)

# Navigation and Role Selection
nav_col1, nav_col2, nav_col3 = st.columns([2, 2, 1])

with nav_col1:
    st.markdown('<div class="nav-container">', unsafe_allow_html=True)
    user_role = st.selectbox(
        "👤 Select Your Role:",
        ['Executive', 'Manager', 'Analyst', 'Sales Director'],
        index=['Executive', 'Manager', 'Analyst', 'Sales Director'].index(st.session_state.user_role)
    )
    st.session_state.user_role = user_role
    st.markdown('</div>', unsafe_allow_html=True)

with nav_col2:
    st.markdown('<div class="nav-container">', unsafe_allow_html=True)
    
    # Role-based page options
    if user_role == 'Executive':
        page_options = ['Dashboard Overview', 'Strategic Metrics', 'Board Report']
    elif user_role == 'Manager':
        page_options = ['Dashboard Overview', 'Team Performance', 'Operations', 'Budget Analysis']
    elif user_role == 'Analyst':
        page_options = ['Dashboard Overview', 'Data Explorer', 'Custom Reports', 'Export Center']
    else:  # Sales Director
        page_options = ['Dashboard Overview', 'Sales Performance', 'Territory Analysis', 'Customer Insights']
    
    selected_page = st.selectbox(
        "📊 Navigate to:",
        page_options
    )
    st.session_state.selected_page = selected_page
    st.markdown('</div>', unsafe_allow_html=True)

with nav_col3:
    st.markdown('<div class="nav-container">', unsafe_allow_html=True)
    st.markdown(f"**🕒 Last Updated**<br>{datetime.now().strftime('%H:%M:%S')}", unsafe_allow_html=True)
    if st.button("🔄 Refresh"):
        st.cache_data.clear()
        st.rerun()
    st.markdown('</div>', unsafe_allow_html=True)

# Page Content Based on Selection
st.markdown("---")

# Calculate current period metrics
current_month = main_df.tail(30)
previous_month = main_df.iloc[-60:-30]

total_orders_current = current_month['orders'].sum()
total_orders_previous = previous_month['orders'].sum()
orders_growth = ((total_orders_current - total_orders_previous) / total_orders_previous * 100)

total_revenue_current = current_month['revenue'].sum()
total_revenue_previous = previous_month['revenue'].sum()
revenue_growth = ((total_revenue_current - total_revenue_previous) / total_revenue_previous * 100)

# Dashboard Overview (Common to all roles)
if selected_page == 'Dashboard Overview':
    st.subheader(f"📊 {user_role} Dashboard Overview")
    
    # Role-specific KPIs
    if user_role == 'Executive':
        kpi_col1, kpi_col2, kpi_col3, kpi_col4, kpi_col5 = st.columns(5)
        
        with kpi_col1:
            st.metric(
                "💰 Monthly Revenue",
                f"R$ {total_revenue_current:,.0f}",
                f"{revenue_growth:+.1f}%"
            )
        
        with kpi_col2:
            avg_satisfaction = current_month['customer_satisfaction'].mean()
            st.metric(
                "⭐ Customer Satisfaction",
                f"{avg_satisfaction:.2f}/5.0",
                "Target: 4.0+"
            )
        
        with kpi_col3:
            market_share = 23.5  # Simulated
            st.metric(
                "📈 Market Share",
                f"{market_share:.1f}%",
                "+1.2%"
            )
        
        with kpi_col4:
            customer_ltv = 485.50  # Simulated
            st.metric(
                "👥 Customer LTV",
                f"R$ {customer_ltv:.0f}",
                "+5.3%"
            )
        
        with kpi_col5:
            operating_margin = 18.7  # Simulated
            st.metric(
                "💼 Operating Margin",
                f"{operating_margin:.1f}%",
                "+0.8%"
            )
    
    elif user_role == 'Manager':
        kpi_col1, kpi_col2, kpi_col3, kpi_col4 = st.columns(4)
        
        with kpi_col1:
            st.metric(
                "📦 Daily Orders",
                f"{current_month['orders'].iloc[-1]:,}",
                f"{orders_growth:+.1f}%"
            )
        
        with kpi_col2:
            fulfillment_rate = 98.5  # Simulated
            st.metric(
                "🚚 Fulfillment Rate",
                f"{fulfillment_rate:.1f}%",
                "Target: 95%+"
            )
        
        with kpi_col3:
            team_productivity = 87.3  # Simulated
            st.metric(
                "👨‍💼 Team Productivity",
                f"{team_productivity:.1f}%",
                "+2.1%"
            )
        
        with kpi_col4:
            budget_utilization = 94.2  # Simulated
            st.metric(
                "💰 Budget Utilization",
                f"{budget_utilization:.1f}%",
                "On Track"
            )
    
    # Status Dashboard
    st.subheader("🚨 System Status & Alerts")
    
    status_col1, status_col2, status_col3 = st.columns(3)
    
    with status_col1:
        if avg_satisfaction >= 4.0:
            st.markdown("""
            <div class="status-green">
                <strong>✅ Customer Satisfaction</strong><br>
                Above target threshold
            </div>
            """, unsafe_allow_html=True)
        else:
            st.markdown("""
            <div class="status-yellow">
                <strong>⚠️ Customer Satisfaction</strong><br>
                Below target - attention needed
            </div>
            """, unsafe_allow_html=True)
    
    with status_col2:
        if revenue_growth > 0:
            st.markdown("""
            <div class="status-green">
                <strong>📈 Revenue Growth</strong><br>
                Positive month-over-month growth
            </div>
            """, unsafe_allow_html=True)
        else:
            st.markdown("""
            <div class="status-red">
                <strong>📉 Revenue Decline</strong><br>
                Immediate attention required
            </div>
            """, unsafe_allow_html=True)
    
    with status_col3:
        system_health = 99.2  # Simulated
        if system_health >= 99:
            st.markdown("""
            <div class="status-green">
                <strong>🔧 System Health</strong><br>
                All systems operational
            </div>
            """, unsafe_allow_html=True)
        else:
            st.markdown("""
            <div class="status-yellow">
                <strong>⚠️ System Issues</strong><br>
                Minor performance degradation
            </div>
            """, unsafe_allow_html=True)
    
    # Key Performance Charts
    st.subheader("📈 Performance Analytics")
    
    chart_col1, chart_col2 = st.columns(2)
    
    with chart_col1:
        # Revenue trend
        monthly_data = main_df.groupby(main_df['date'].dt.to_period('M')).agg({
            'revenue': 'sum',
            'orders': 'sum'
        }).reset_index()
        monthly_data['date'] = monthly_data['date'].astype(str)
        
        fig = px.line(
            monthly_data,
            x='date',
            y='revenue',
            title="Monthly Revenue Trend",
            labels={'revenue': 'Revenue (R$)', 'date': 'Month'}
        )
        fig.update_traces(line=dict(width=3, color='#667eea'))
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)
    
    with chart_col2:
        # Department performance
        latest_dept = dept_df.groupby('department')['target_achievement'].mean().reset_index()
        
        fig = px.bar(
            latest_dept.sort_values('target_achievement', ascending=True),
            x='target_achievement',
            y='department',
            orientation='h',
            title="Department Target Achievement",
            labels={'target_achievement': 'Target Achievement (%)', 'department': 'Department'},
            color='target_achievement',
            color_continuous_scale='RdYlGn'
        )
        fig.add_vline(x=100, line_dash="dash", line_color="red", 
                     annotation_text="Target")
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

# Role-specific pages
elif selected_page == 'Strategic Metrics' and user_role == 'Executive':
    st.subheader("🎯 Strategic Business Metrics")
    
    # Executive-level strategic dashboard
    strategy_col1, strategy_col2 = st.columns(2)
    
    with strategy_col1:
        # Customer acquisition funnel
        funnel_data = {
            'Stage': ['Visitors', 'Leads', 'Prospects', 'Customers', 'Repeat Customers'],
            'Count': [100000, 15000, 5000, 1900, 1200],
            'Conversion': [100, 15, 5, 1.9, 1.2]
        }
        funnel_df = pd.DataFrame(funnel_data)
        
        fig = go.Figure(go.Funnel(
            y = funnel_df['Stage'],
            x = funnel_df['Count'],
            textinfo = "value+percent initial"
        ))
        fig.update_layout(title="Customer Acquisition Funnel", height=400)
        st.plotly_chart(fig, use_container_width=True)
    
    with strategy_col2:
        # Market position analysis
        competitors = ['Olist', 'Mercado Livre', 'Amazon Brasil', 'Magazine Luiza', 'Others']
        market_share = [23.5, 35.2, 18.7, 12.1, 10.5]
        
        fig = px.pie(
            values=market_share,
            names=competitors,
            title="Brazilian E-commerce Market Share",
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig.update_layout(height=400)
        st.plotly_chart(fig, use_container_width=True)

elif selected_page == 'Team Performance' and user_role == 'Manager':
    st.subheader("👨‍💼 Team Performance Dashboard")
    
    # Team performance metrics
    team_data = dept_df.groupby('department').agg({
        'performance_score': 'mean',
        'target_achievement': 'mean',
        'team_size': 'first',
        'budget_utilization': 'mean'
    }).reset_index()
    
    st.dataframe(
        team_data.style.format({
            'performance_score': '{:.2f}',
            'target_achievement': '{:.1f}%',
            'budget_utilization': '{:.1f}%'
        }),
        use_container_width=True
    )

elif selected_page == 'Data Explorer' and user_role == 'Analyst':
    st.subheader("🔍 Advanced Data Explorer")
    
    # Advanced filtering interface
    filter_col1, filter_col2, filter_col3 = st.columns(3)
    
    with filter_col1:
        date_range = st.date_input(
            "Date Range:",
            value=[main_df['date'].min().date(), main_df['date'].max().date()],
            min_value=main_df['date'].min().date(),
            max_value=main_df['date'].max().date()
        )
    
    with filter_col2:
        metric_selection = st.multiselect(
            "Metrics to Analyze:",
            ['orders', 'revenue', 'customer_satisfaction', 'conversion_rate'],
            default=['orders', 'revenue']
        )
    
    with filter_col3:
        aggregation = st.selectbox(
            "Aggregation Level:",
            ['Daily', 'Weekly', 'Monthly']
        )
    
    # Display filtered data
    if len(date_range) == 2:
        filtered_data = main_df[
            (main_df['date'].dt.date >= date_range[0]) &
            (main_df['date'].dt.date <= date_range[1])
        ]
        
        st.dataframe(filtered_data[['date'] + metric_selection], use_container_width=True)

elif selected_page == 'Sales Performance' and user_role == 'Sales Director':
    st.subheader("💼 Sales Performance Analytics")
    
    # Sales-specific metrics
    sales_col1, sales_col2, sales_col3 = st.columns(3)
    
    with sales_col1:
        quota_achievement = 112.5  # Simulated
        st.metric(
            "🎯 Quota Achievement",
            f"{quota_achievement:.1f}%",
            "+12.5%"
        )
    
    with sales_col2:
        pipeline_value = 2850000  # Simulated
        st.metric(
            "💰 Pipeline Value",
            f"R$ {pipeline_value:,.0f}",
            "Q4 Target"
        )
    
    with sales_col3:
        win_rate = 23.8  # Simulated
        st.metric(
            "🏆 Win Rate",
            f"{win_rate:.1f}%",
            "+1.2%"
        )

# Sidebar Information
with st.sidebar:
    st.header("ℹ️ Dashboard Information")
    
    st.markdown(f"""
    **Current User**: {user_role}  
    **Active Page**: {selected_page}  
    **Data Period**: Last 6 months  
    **Update Frequency**: Real-time  
    **Last Refresh**: {datetime.now().strftime('%H:%M:%S')}
    """)
    
    st.markdown("---")
    st.subheader("📊 Quick Actions")
    
    if st.button("📄 Export Report"):
        st.success("✅ Report exported successfully!")
    
    if st.button("📧 Email Summary"):
        st.success("✅ Summary sent to stakeholders!")
    
    if st.button("⚙️ Settings"):
        st.info("Settings panel would open here")
    
    st.markdown("---")
    st.subheader("🔗 Quick Links")
    st.markdown("""
    - [📊 Full Analytics](https://analytics.olist.com)
    - [📈 Financial Reports](https://finance.olist.com)
    - [👥 HR Dashboard](https://hr.olist.com)
    - [🛠️ Admin Panel](https://admin.olist.com)
    """)

# Footer
st.markdown("""
<div class="footer">
    <p><strong>🏢 Olist Business Intelligence Platform v2.1</strong></p>
    <p>Built with Streamlit | Real-time Supabase Integration | 
    <a href="#" style="color: #667eea;">Support</a> | 
    <a href="#" style="color: #667eea;">Documentation</a> | 
    <a href="#" style="color: #667eea;">API</a></p>
</div>
""", unsafe_allow_html=True)

## 🎨 Section 2: Advanced Layout Patterns (7 minutes)

Let's explore sophisticated layout techniques for professional business applications:

In [ ]:
%%writefile advanced_layouts_app.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Configure page
st.set_page_config(
    page_title="Advanced Layout Patterns",
    page_icon="🎨",
    layout="wide"
)

# Advanced CSS for responsive design
st.markdown("""
<style>
    /* Responsive Grid System */
    .responsive-grid {
        display: grid;
        grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
        gap: 1rem;
        margin: 1rem 0;
    }
    
    /* Card Components */
    .metric-card {
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
        padding: 1.5rem;
        border-radius: 12px;
        border: 1px solid #e1e8ed;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.07);
        transition: transform 0.2s ease;
    }
    
    .metric-card:hover {
        transform: translateY(-2px);
        box-shadow: 0 8px 15px rgba(0, 0, 0, 0.1);
    }
    
    /* Dashboard Sections */
    .dashboard-section {
        background: white;
        padding: 2rem;
        border-radius: 10px;
        border: 1px solid #e0e6ed;
        margin: 1rem 0;
        position: relative;
    }
    
    .section-header {
        background: linear-gradient(90deg, #667eea, #764ba2);
        color: white;
        padding: 1rem 2rem;
        margin: -2rem -2rem 1rem -2rem;
        border-radius: 10px 10px 0 0;
        font-weight: bold;
        font-size: 1.1rem;
    }
    
    /* Sidebar Styling */
    .sidebar-content {
        background: #f8f9fa;
        padding: 1rem;
        border-radius: 8px;
        border-left: 4px solid #667eea;
        margin: 1rem 0;
    }
    
    /* Status Indicators */
    .status-indicator {
        display: inline-block;
        width: 12px;
        height: 12px;
        border-radius: 50%;
        margin-right: 8px;
    }
    
    .status-online { background-color: #28a745; }
    .status-warning { background-color: #ffc107; }
    .status-offline { background-color: #dc3545; }
    
    /* Mobile Responsive */
    @media (max-width: 768px) {
        .responsive-grid {
            grid-template-columns: 1fr;
        }
        
        .dashboard-section {
            padding: 1rem;
        }
        
        .section-header {
            margin: -1rem -1rem 1rem -1rem;
            padding: 0.75rem 1rem;
        }
    }
    
    /* Animation Effects */
    .fade-in {
        animation: fadeIn 0.5s ease-in;
    }
    
    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(20px); }
        to { opacity: 1; transform: translateY(0); }
    }
</style>
""", unsafe_allow_html=True)

st.title("🎨 Advanced Layout Patterns for Business Dashboards")
st.markdown("**Professional design patterns for enterprise applications**")

# Generate sample data
@st.cache_data
def generate_layout_demo_data():
    np.random.seed(42)
    
    # KPI data
    kpis = {
        'revenue': {'value': 2850000, 'change': 15.3, 'status': 'up'},
        'orders': {'value': 45230, 'change': 8.7, 'status': 'up'},
        'customers': {'value': 12450, 'change': -2.1, 'status': 'down'},
        'satisfaction': {'value': 4.25, 'change': 0.15, 'status': 'up'},
        'conversion': {'value': 3.8, 'change': 0.3, 'status': 'up'},
        'avg_order': {'value': 125.50, 'change': 5.2, 'status': 'up'}
    }
    
    # Time series data
    dates = pd.date_range('2024-01-01', periods=180, freq='D')
    ts_data = pd.DataFrame({
        'date': dates,
        'revenue': np.random.normal(15000, 3000, 180).cumsum(),
        'orders': np.random.poisson(150, 180),
        'users': np.random.poisson(500, 180)
    })
    
    return kpis, ts_data

kpis, ts_data = generate_layout_demo_data()

# Layout Pattern 1: Grid-based KPI Cards
st.markdown("""
<div class="dashboard-section fade-in">
    <div class="section-header">📊 Pattern 1: Responsive KPI Grid</div>
</div>
""", unsafe_allow_html=True)

# Create responsive grid with custom HTML/CSS
kpi_grid_html = """
<div class="responsive-grid">
"""

kpi_items = [
    ('💰', 'Revenue', f"R$ {kpis['revenue']['value']:,}", f"+{kpis['revenue']['change']:.1f}%"),
    ('📦', 'Orders', f"{kpis['orders']['value']:,}", f"+{kpis['orders']['change']:.1f}%"),
    ('👥', 'Customers', f"{kpis['customers']['value']:,}", f"{kpis['customers']['change']:+.1f}%"),
    ('⭐', 'Satisfaction', f"{kpis['satisfaction']['value']:.2f}/5.0", f"+{kpis['satisfaction']['change']:.2f}"),
    ('🎯', 'Conversion', f"{kpis['conversion']['value']:.1f}%", f"+{kpis['conversion']['change']:.1f}%"),
    ('💳', 'Avg Order', f"R$ {kpis['avg_order']['value']:.2f}", f"+{kpis['avg_order']['change']:.1f}%")
]

for icon, label, value, change in kpi_items:
    color = "#28a745" if '+' in change else "#dc3545"
    kpi_grid_html += f"""
    <div class="metric-card">
        <div style="font-size: 2rem; margin-bottom: 0.5rem;">{icon}</div>
        <div style="font-size: 0.9rem; color: #666; margin-bottom: 0.25rem;">{label}</div>
        <div style="font-size: 1.5rem; font-weight: bold; margin-bottom: 0.25rem;">{value}</div>
        <div style="font-size: 0.8rem; color: {color}; font-weight: bold;">{change}</div>
    </div>
    """

kpi_grid_html += "</div>"
st.markdown(kpi_grid_html, unsafe_allow_html=True)

# Layout Pattern 2: Tabbed Interface with Charts
st.markdown("""
<div class="dashboard-section fade-in">
    <div class="section-header">📈 Pattern 2: Multi-Tab Analytics</div>
</div>
""", unsafe_allow_html=True)

tab1, tab2, tab3, tab4 = st.tabs(["📊 Overview", "💰 Revenue", "📦 Orders", "👥 Users"])

with tab1:
    overview_col1, overview_col2 = st.columns(2)
    
    with overview_col1:
        # Combined metrics chart
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=ts_data['date'],
            y=ts_data['revenue'],
            mode='lines',
            name='Revenue',
            line=dict(color='#667eea', width=3)
        ))
        
        fig.update_layout(
            title="Revenue Trend Overview",
            height=350,
            showlegend=False
        )
        
        st.plotly_chart(fig, use_container_width=True)
    
    with overview_col2:
        # Status summary
        st.markdown("""
        <div style="padding: 1rem; background: #f8f9fa; border-radius: 8px;">
            <h4>🚦 System Status</h4>
            <p><span class="status-indicator status-online"></span>Database: Online</p>
            <p><span class="status-indicator status-online"></span>API: Operational</p>
            <p><span class="status-indicator status-warning"></span>Cache: Degraded</p>
            <p><span class="status-indicator status-online"></span>Payments: Active</p>
        </div>
        """, unsafe_allow_html=True)
        
        # Quick actions
        st.markdown("### ⚡ Quick Actions")
        
        action_col1, action_col2 = st.columns(2)
        
        with action_col1:
            if st.button("📊 Generate Report", use_container_width=True):
                st.success("Report generated!")
        
        with action_col2:
            if st.button("🔄 Refresh Data", use_container_width=True):
                st.success("Data refreshed!")

with tab2:
    # Revenue analytics
    revenue_col1, revenue_col2 = st.columns([2, 1])
    
    with revenue_col1:
        # Monthly revenue breakdown
        monthly_revenue = ts_data.groupby(ts_data['date'].dt.to_period('M'))['revenue'].sum().reset_index()
        monthly_revenue['date'] = monthly_revenue['date'].astype(str)
        
        fig = px.bar(
            monthly_revenue,
            x='date',
            y='revenue',
            title="Monthly Revenue Performance",
            color='revenue',
            color_continuous_scale='Blues'
        )
        
        st.plotly_chart(fig, use_container_width=True)
    
    with revenue_col2:
        # Revenue statistics
        st.metric("📈 Total Revenue", f"R$ {ts_data['revenue'].sum():,.0f}")
        st.metric("📊 Average Daily", f"R$ {ts_data['revenue'].mean():,.0f}")
        st.metric("🔝 Best Day", f"R$ {ts_data['revenue'].max():,.0f}")
        st.metric("📉 Lowest Day", f"R$ {ts_data['revenue'].min():,.0f}")

with tab3:
    # Orders analytics
    st.subheader("📦 Order Analysis")
    
    # Daily orders trend
    fig = px.line(
        ts_data,
        x='date',
        y='orders',
        title="Daily Orders Trend",
        line_shape='spline'
    )
    
    fig.update_traces(line=dict(color='#764ba2', width=2))
    st.plotly_chart(fig, use_container_width=True)

with tab4:
    # User analytics
    st.subheader("👥 User Engagement")
    
    # User activity heatmap
    ts_data['day_of_week'] = ts_data['date'].dt.day_name()
    ts_data['week'] = ts_data['date'].dt.isocalendar().week
    
    pivot_data = ts_data.pivot_table(
        values='users',
        index='day_of_week',
        columns='week',
        aggfunc='mean'
    )
    
    fig = px.imshow(
        pivot_data,
        title="User Activity Heatmap (Day vs Week)",
        color_continuous_scale='Viridis'
    )
    
    st.plotly_chart(fig, use_container_width=True)

# Layout Pattern 3: Expandable Sections
st.markdown("""
<div class="dashboard-section fade-in">
    <div class="section-header">🔽 Pattern 3: Expandable Content Sections</div>
</div>
""", unsafe_allow_html=True)

with st.expander("📊 Detailed Analytics", expanded=False):
    detail_col1, detail_col2, detail_col3 = st.columns(3)
    
    with detail_col1:
        st.markdown("**📈 Growth Metrics**")
        st.write("• Monthly Growth: +15.3%")
        st.write("• YoY Growth: +45.7%")
        st.write("• Quarterly Target: 125%")
    
    with detail_col2:
        st.markdown("**👥 Customer Insights**")
        st.write("• New Customers: 2,340")
        st.write("• Retention Rate: 87.5%")
        st.write("• Average LTV: R$ 485")
    
    with detail_col3:
        st.markdown("**⚙️ Operational Metrics**")
        st.write("• Fulfillment Rate: 98.5%")
        st.write("• Average Processing: 2.3hrs")
        st.write("• Return Rate: 2.1%")

with st.expander("🔧 Configuration Options", expanded=False):
    config_col1, config_col2 = st.columns(2)
    
    with config_col1:
        st.selectbox("Dashboard Theme:", ['Professional', 'Dark', 'Colorful', 'Minimal'])
        st.selectbox("Update Frequency:", ['Real-time', '5 minutes', '15 minutes', 'Hourly'])
        st.checkbox("Enable Notifications", value=True)
    
    with config_col2:
        st.selectbox("Export Format:", ['PDF', 'Excel', 'CSV', 'PowerPoint'])
        st.selectbox("Email Reports:", ['Daily', 'Weekly', 'Monthly', 'Never'])
        st.checkbox("Mobile Optimized", value=True)

# Layout Pattern 4: Sidebar Integration
with st.sidebar:
    st.markdown("""
    <div class="sidebar-content">
        <h3>🎛️ Dashboard Controls</h3>
    </div>
    """, unsafe_allow_html=True)
    
    # Time range selector
    time_range = st.selectbox(
        "📅 Time Range:",
        ['Last 7 days', 'Last 30 days', 'Last 90 days', 'Custom range']
    )
    
    # Filter controls
    st.markdown("### 🔍 Filters")
    
    department = st.multiselect(
        "Department:",
        ['Sales', 'Marketing', 'Operations', 'Finance'],
        default=['Sales', 'Marketing']
    )
    
    region = st.selectbox(
        "Region:",
        ['All Regions', 'North', 'South', 'East', 'West', 'Central']
    )
    
    # Performance threshold
    performance_threshold = st.slider(
        "Performance Threshold:",
        min_value=0,
        max_value=100,
        value=75,
        help="Show only items above this performance level"
    )
    
    st.markdown("---")
    
    # Export section
    st.markdown("""
    <div class="sidebar-content">
        <h3>💾 Export & Share</h3>
    </div>
    """, unsafe_allow_html=True)
    
    if st.button("📊 Export Dashboard", use_container_width=True):
        st.success("Dashboard exported!")
    
    if st.button("📧 Email Report", use_container_width=True):
        st.success("Report emailed!")
    
    if st.button("🔗 Share Link", use_container_width=True):
        st.info("Share link copied to clipboard!")
    
    st.markdown("---")
    
    # Help section
    st.markdown("""
    <div class="sidebar-content">
        <h3>❓ Help & Support</h3>
        <p><a href="#">📚 Documentation</a></p>
        <p><a href="#">🎥 Video Tutorials</a></p>
        <p><a href="#">💬 Contact Support</a></p>
        <p><a href="#">🐛 Report Bug</a></p>
    </div>
    """, unsafe_allow_html=True)

# Footer with additional information
st.markdown("---")
st.markdown("""
<div style="text-align: center; color: #666; padding: 1rem;">
    <p><strong>🎨 Advanced Layout Patterns</strong> | 
    Enterprise-grade dashboard design | 
    Responsive & Mobile-optimized</p>
    <p>Built with Streamlit | Last updated: {}</p>
</div>
""".format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')), unsafe_allow_html=True)

## 📱 Section 3: Responsive Design Considerations (5 minutes)

Modern business dashboards must work across devices. Here are key responsive design principles:

In [ ]:
# Responsive design best practices
import streamlit as st

st.subheader("📱 Responsive Design Best Practices")

st.markdown("""
### 1. Mobile-First Design Principles

**✅ Do:**
- Use `st.columns()` with responsive breakpoints
- Design for touch interactions (larger buttons)
- Prioritize essential information
- Test on multiple screen sizes

**❌ Avoid:**
- Fixed pixel widths
- Tiny clickable elements
- Horizontal scrolling
- Overcrowded layouts

### 2. Adaptive Column Layouts

```python
# Desktop: 4 columns, Tablet: 2 columns, Mobile: 1 column
col1, col2, col3, col4 = st.columns([1, 1, 1, 1])

# Use containers for better mobile stacking
with st.container():
    desktop_cols = st.columns(4)
    for i, col in enumerate(desktop_cols):
        with col:
            st.metric(f"KPI {i+1}", f"{i*100:,}")
```

### 3. Progressive Disclosure

```python
# Show essential info first, details in expanders
st.metric("Revenue", "R$ 1.2M", "+15%")

with st.expander("📊 Detailed Breakdown"):
    # Detailed charts and data tables here
    pass
```

### 4. Touch-Friendly Interactions

```python
# Use larger buttons for mobile
if st.button("💰 View Revenue Details", use_container_width=True):
    # Action here
    pass

# Prefer selectboxes over multi-step dropdowns
category = st.selectbox("Category:", options, key="mobile_category")
```

### 5. Performance Optimization

```python
# Conditional loading for mobile
if st.checkbox("Show Advanced Charts"):
    # Only load heavy visualizations when requested
    create_complex_chart()
```
""")

# Responsive demo
st.markdown("---")
st.subheader("📱 Responsive Layout Demo")

# This layout will automatically adapt to screen size
responsive_col1, responsive_col2, responsive_col3 = st.columns([2, 1, 1])

with responsive_col1:
    st.markdown("""
    **Main Content Area**
    
    This column takes 50% width on desktop but will stack vertically on mobile devices.
    
    Key content should go here since it appears first on mobile.
    """)

with responsive_col2:
    st.metric("📊 Metric 1", "1,234", "+5%")
    
with responsive_col3:
    st.metric("📈 Metric 2", "5,678", "+12%")

# Mobile optimization tips
with st.expander("📱 Mobile Optimization Tips"):
    st.markdown("""
    **For Executive Users on Mobile:**
    - Show only critical KPIs above the fold
    - Use large, easy-to-read fonts
    - Implement swipe gestures for navigation
    - Provide offline capability for key metrics
    
    **For Analyst Users on Mobile:**
    - Enable data export for desktop analysis
    - Provide simplified filtering options
    - Show data summaries instead of full tables
    - Allow drilling down to detailed views
    """)

print("✅ Responsive design principles covered!")

## 🎯 Key Takeaways

✅ **Multi-Page Architecture**: Role-based navigation and user interfaces  
✅ **Advanced Layouts**: Grid systems, responsive design, professional styling  
✅ **Enterprise Features**: Status indicators, quick actions, configuration options  
✅ **Mobile Optimization**: Touch-friendly interfaces and adaptive layouts  
✅ **Professional Branding**: Corporate styling and consistent design language  

## 🔜 Coming Up in Part 2

In the next session, we'll dive deep into database integration and real-time data handling:

**Preview Topics:**
- Advanced Supabase integration patterns
- Real-time data streaming
- Connection pooling and optimization
- Error handling and fallback strategies

---

## 💼 Practice Exercise

**Scenario**: Create a multi-page executive dashboard for Olist's quarterly board meeting.

**Requirements**:
1. **Landing Page**: Executive summary with key KPIs
2. **Financial Page**: Revenue, profit margins, and growth metrics
3. **Operations Page**: Fulfillment, customer service, and logistics
4. **Strategy Page**: Market position and competitive analysis

**Design Constraints**:
- Professional corporate styling
- Mobile-responsive layout
- Role-based access (CEO vs CFO views)
- Export capabilities for each page

**Time**: 15 minutes

---

*Next: [Part 2 - Database Integration →](02_streamlit_advanced_part2_database_integration.ipynb)*